In [1]:
# Imports 
import pandas as pd
import sqlalchemy

In [ ]:
# Create SQLAlchemy engine for connecting to SQLite database
engine = sqlalchemy.create_engine("sqlite:///test.db?journal_mode=DELETE&synchronous=NORMAL&temp_store=MEMORY")

In [3]:
# Data Path
path = 'Data/POS_Data.xlsx'

# Retrieve Data
raw_data = pd.read_excel(path, sheet_name='POS')

raw_data.head()

,Check ID,Item Name,Gross Revenue,Date,Sale Time - Exact,Category,Day Part,Is Beverage on Check,Cost Center
0,5723,BVC - WATER SMARTWATER 20OZ,1.59,2020-11-20,21:22:30,ADD>BreakfstAdds,Late Night,Yes,Hospital B
1,27172,BVC - WATER SMARTWATER 20OZ,0.00,2020-10-28,12:47:50,ADD>BreakfstAdds,Lunch,Yes,Hospital A
2,34799,BVC - WATER SMARTWATER 20OZ,1.59,2020-10-28,07:15:04,ADD>BreakfstAdds,Breakfast,Yes,Hospital A
3,64634,BVC - WATER SMARTWATER 20OZ,1.59,2020-10-27,08:39:39,ADD>BreakfstAdds,Breakfast,Yes,Hospital A
4,64634,BVC - WATER SMARTWATER 20OZ,1.59,2020-10-27,08:39:39,ADD>BreakfstAdds,Mid Morning,Yes,Hospital A


In [4]:
# Clean Column Names
raw_data.columns = (
    raw_data.columns
    .str.strip()
    .str.lower()
    .str.replace(' ', '_')
    .str.replace('-', '_')
    .str.replace('___', '_')
    .str.replace('__', '_')
    .str.replace('-', '_')
    .str.replace(r'[^a-z0-9_]', '', regex=True)
)

raw_data.head()

,check_id,item_name,gross_revenue,date,sale_time_exact,category,day_part,is_beverage_on_check,cost_center
0,5723,BVC - WATER SMARTWATER 20OZ,1.59,2020-11-20,21:22:30,ADD>BreakfstAdds,Late Night,Yes,Hospital B
1,27172,BVC - WATER SMARTWATER 20OZ,0.00,2020-10-28,12:47:50,ADD>BreakfstAdds,Lunch,Yes,Hospital A
2,34799,BVC - WATER SMARTWATER 20OZ,1.59,2020-10-28,07:15:04,ADD>BreakfstAdds,Breakfast,Yes,Hospital A
3,64634,BVC - WATER SMARTWATER 20OZ,1.59,2020-10-27,08:39:39,ADD>BreakfstAdds,Breakfast,Yes,Hospital A
4,64634,BVC - WATER SMARTWATER 20OZ,1.59,2020-10-27,08:39:39,ADD>BreakfstAdds,Mid Morning,Yes,Hospital A


In [5]:
raw_data['group'] = raw_data['item_name'].str.split(' - ', n=1).str[0]
raw_data['item_name'] = raw_data['item_name'].str.split(' - ', n=1).str[-1]
raw_data['sub_category'] = raw_data['category'].str.split('>').str[-1]
raw_data['category'] = raw_data['category'].str.split('>').str[0]

line_items = raw_data.assign(
    timestamp=lambda df: pd.to_datetime(raw_data['date'].astype(str) + ' ' + raw_data['sale_time_exact'].astype(str), format='%Y-%m-%d %H:%M:%S', errors='coerce'),
)

# Drop unneeded date/time columns
line_items = line_items.drop(columns=['date', 'sale_time_exact'])

line_items.head()

,check_id,item_name,gross_revenue,category,day_part,is_beverage_on_check,cost_center,group,sub_category,timestamp
0,5723,WATER SMARTWATER 20OZ,1.59,ADD,Late Night,Yes,Hospital B,BVC,BreakfstAdds,2020-11-20 21:22:30
1,27172,WATER SMARTWATER 20OZ,0.00,ADD,Lunch,Yes,Hospital A,BVC,BreakfstAdds,2020-10-28 12:47:50
2,34799,WATER SMARTWATER 20OZ,1.59,ADD,Breakfast,Yes,Hospital A,BVC,BreakfstAdds,2020-10-28 07:15:04
3,64634,WATER SMARTWATER 20OZ,1.59,ADD,Breakfast,Yes,Hospital A,BVC,BreakfstAdds,2020-10-27 08:39:39
4,64634,WATER SMARTWATER 20OZ,1.59,ADD,Mid Morning,Yes,Hospital A,BVC,BreakfstAdds,2020-10-27 08:39:39


In [6]:
line_items['is_beverage_on_check'] = line_items['is_beverage_on_check'].str.lower().str.strip() == 'yes'

line_items = line_items.astype({
    'check_id': 'string',
    'item_name': 'string',
    'gross_revenue': 'float',
    'category': 'string',
    'sub_category': 'string',
    'day_part': 'string',
    'is_beverage_on_check': 'boolean',
    'cost_center': 'string',
    'group': 'string',
    'timestamp': 'datetime64[ns]'
})


# Add the primary key column (1-based index)
line_items['line_item_id'] = line_items.index + 1


# Load line items to SQLite database
line_items.to_sql("line_items", engine, if_exists="replace", index=True, chunksize=8192, method="multi")


line_items.head()

,check_id,item_name,gross_revenue,category,day_part,is_beverage_on_check,cost_center,group,sub_category,timestamp,line_item_id
0,5723,WATER SMARTWATER 20OZ,1.59,ADD,Late Night,True,Hospital B,BVC,BreakfstAdds,2020-11-20 21:22:30,1
1,27172,WATER SMARTWATER 20OZ,0.00,ADD,Lunch,True,Hospital A,BVC,BreakfstAdds,2020-10-28 12:47:50,2
2,34799,WATER SMARTWATER 20OZ,1.59,ADD,Breakfast,True,Hospital A,BVC,BreakfstAdds,2020-10-28 07:15:04,3
3,64634,WATER SMARTWATER 20OZ,1.59,ADD,Breakfast,True,Hospital A,BVC,BreakfstAdds,2020-10-27 08:39:39,4
4,64634,WATER SMARTWATER 20OZ,1.59,ADD,Mid Morning,True,Hospital A,BVC,BreakfstAdds,2020-10-27 08:39:39,5


In [7]:
# build transactions table and enforce data types
transactions = line_items.groupby(['check_id']).agg(
    timestamp = ('timestamp', 'first'),
    total_amount=('gross_revenue', 'sum'),
    num_items=('item_name', 'count'),
    cost_center=('cost_center', 'first'),
    day_part=('day_part', 'first'),
    top_group = ('group', lambda x: x.mode()[0]),
    is_beverage_on_check = ('is_beverage_on_check', 'first')
    ).reset_index().astype({
        'check_id': 'string',
        'timestamp': 'datetime64[ns]',
        'total_amount': 'float64',
        'num_items': 'Int64',
        'cost_center': 'string',
        'day_part': 'string',
        'top_group': 'string',
    })

# # Add the primary key column (1-based index)
# transactions['transactions_id'] = transactions.index + 1

# Load transactions table to SQLite database
transactions.to_sql("transactions", engine, if_exists="replace", index=True, chunksize=8192, method="multi")

transactions.head()

,check_id,timestamp,total_amount,num_items,cost_center,day_part,top_group,is_beverage_on_check
0,10000003,2020-10-23 11:58:15,4.09,1,Hospital A,Lunch,GRL,False
1,10000360,2020-10-23 13:05:04,9.66,4,Hospital B,Lunch,GRL,True
2,10000365,2020-10-23 12:33:39,5.49,1,Hospital B,Lunch,TUKEY PROVOLONE,False
3,10000385,2020-10-23 12:41:35,12.46,5,Hospital A,Lunch,GRL,True
4,10000390,2020-10-23 13:15:30,0.99,1,Hospital A,Lunch,BFK,False


In [10]:
# Build items table
items = line_items[['item_name', 'group', 'category', 'sub_category', 'gross_revenue', 'cost_center']].drop_duplicates().sort_values('item_name').reset_index(drop=True)
items = items.astype({
    'item_name': 'string',
    'group': 'string',
    'category': 'string',
    'sub_category': 'string',
    'gross_revenue': 'float64',
    'cost_center': 'string'
}) 

items = items.rename(columns={'gross_revenue': 'price'})

# Load items table to SQLite database
items.to_sql("items", engine, if_exists="replace", index=True, chunksize=8192, method="multi")

items.head()

,item_name,group,category,sub_category,price,cost_center
0,CAFE KITCHEN CHICKEN,ENT,Entree,Entree,6.99,Hospital A
1,CAFE KITCHEN CHICKEN,ENT,Entree,Entree,0.00,Hospital A
2,CAFE KITCHEN PASTA,ENT,Entree,Entree,5.99,Hospital A
3,CAFE KITCHEN PASTA,ENT,Entree,Entree,0.00,Hospital A
4,GUEST TRAY 7.50,ENT,Entree,Entree,13.86,Hospital A


In [11]:
#Build categories table
categories = line_items[['category', 'sub_category', 'cost_center']].drop_duplicates().sort_values(['category', 'sub_category', 'cost_center']).reset_index(drop=True)

with pd.option_context('display.max_rows', None,
                    'display.max_columns', None,
                    'display.width', None,
                    'display.max_colwidth', None):
    display(categories)


# Load categories table into SQLite database
categories.to_sql("categories", engine, if_exists="replace", index=True, chunksize=8192, method="multi")

,category,sub_category,cost_center
0,ADD,BreakfstAdds,Hospital A
1,ADD,BreakfstAdds,Hospital B
2,ADD,Coffee Adds,Hospital A
3,ADD,Coffee Adds,Hospital B
4,ADD,Combo Adds,Hospital A
5,ADD,Combo Adds,Hospital B
6,ADD,Pizza Adds,Hospital A
7,ADD,Pizza Adds,Hospital B
8,ADD,Sand Adds,Hospital A
9,ALC,Beer,Hospital A


103

In [1]:
# Basket Combo Pipeline Testing

import pandas as pd
from collections import Counter, defaultdict
import itertools
import sqlalchemy

# Create SQLAlchemy engine
engine = sqlalchemy.create_engine("sqlite:///C1_case_study.db")

fact_line_items = pd.read_sql_table('fact_line_items', engine)
dim_items = pd.read_sql_table('dim_items', engine)


# Join Fact Line Items on Dim Items (to get Name)
line_items_named = fact_line_items.merge(dim_items, on='item_id')

# Group items by Transaction ID to get baskets
baskets = line_items_named.groupby('transaction_id')['item_name'].apply(list)

# Filter for Multi-Item Baskets
multi_item_baskets = baskets[baskets.apply(len) > 1]

# Identify multi-item transaction IDs
multi_tx_ids = multi_item_baskets.index

# Filter line items to only multi-item transactions
multi_line_items = line_items_named[line_items_named['transaction_id'].isin(multi_tx_ids)]

# Find All Pairs with Frequency and Total Revenue
pair_counts = Counter()
pair_revenue = defaultdict(float)
for tx_id in multi_tx_ids:
    tx_lines = multi_line_items[multi_line_items['transaction_id'] == tx_id]
    item_rev = tx_lines.groupby('item_name')['gross_revenue'].sum().to_dict()
    unique_items = sorted(item_rev.keys())
    if len(unique_items) > 1:
        pairs = list(itertools.combinations(unique_items, 2))
        pair_counts.update(pairs)
        for pair in pairs:
            a, b = pair
            pair_revenue[pair] += item_rev.get(a, 0) + item_rev.get(b, 0)

# Create dataframe for pairs
pairs_data = []
for pair, freq in sorted(pair_counts.items(), key=lambda x: pair_revenue[x[0]], reverse=True):
    revenue = pair_revenue[pair]
    if revenue > 0 and freq > 0:
        pairs_data.append({'Pair': pair, 'Frequency': freq, 'Total Revenue': revenue})

all_pairs = pd.DataFrame(pairs_data).sort_values(['Frequency'], ascending=False).reset_index(drop=True)

# To prevent display truncation if there are many rows
pd.set_option('display.max_rows', None)

print("All Combinations with Frequency > 0 and Total Revenue > 0:")
all_pairs.head()

All Combinations with Frequency > 0 and Total Revenue > 0:


,Pair,Frequency,Total Revenue
0,"(ENTREE 4.29, SIDE 1.29)",3659,21212.819696
1,"(BACON 3 SLICES, EGGS)",2348,8250.039975
2,"(ENT -PREMIUM SIDE 1.39, SIDE 1.29)",2195,6195.719878
3,"(ENTREE 4.79, SIDE 1.29)",2171,14391.029803
4,"(EGGS, HOME FRIES)",1961,7442.499915


In [9]:
import pandas as pd
from collections import Counter, defaultdict
import itertools

# Assuming fact_line_items and dim_items are defined elsewhere, and fact_line_items has 'gross_revenue',
# and dim_items has 'item_cost_center' and 'item_name' (assuming item_name is unique)

# Basket/combo analysis

# Join Fact Line Items on Dim Items (to get Name and other fields)
line_items_named = fact_line_items.merge(dim_items, on='item_id')

# Group items by Transaction ID to get baskets
baskets = line_items_named.groupby('transaction_id')['item_name'].apply(list)

# Filter for Multi-Item Baskets
multi_item_baskets = baskets[baskets.apply(len) > 1]

# Identify multi-item transaction IDs
multi_tx_ids = multi_item_baskets.index

# Filter line items to only multi-item transactions
multi_line_items = line_items_named[line_items_named['transaction_id'].isin(multi_tx_ids)]

# Get item to cost center mapping (assuming unique item_name)
item_to_cc = dim_items.set_index('item_name')['item_cost_center'].to_dict()

# Find All Pairs with Frequency and Total Revenue, only if same cost center
pair_counts = Counter()
pair_revenue = defaultdict(float)
for tx_id in multi_tx_ids:
    tx_lines = multi_line_items[multi_line_items['transaction_id'] == tx_id]
    item_rev = tx_lines.groupby('item_name')['gross_revenue'].sum().to_dict()
    unique_items = sorted(item_rev.keys())
    if len(unique_items) > 1:
        for pair in itertools.combinations(unique_items, 2):
            a, b = pair
            cc1 = item_to_cc.get(a)
            cc2 = item_to_cc.get(b)
            if cc1 == cc2 and cc1 is not None:
                pair_counts[pair] += 1
                pair_revenue[pair] += item_rev.get(a, 0) + item_rev.get(b, 0)

# Create dataframe for pairs
pairs_data = []
for pair, freq in sorted(pair_counts.items(), key=lambda x: pair_revenue[x[0]], reverse=True):
    revenue = pair_revenue[pair]
    if revenue > 0 and freq > 0:
        a, b = pair
        cc = item_to_cc.get(a)  # Since same as b
        pairs_data.append({
            'Cost Center': cc,
            'Item1': a,
            'Item2': b,
            'Frequency': freq,
            'Total Revenue': revenue
        })

all_pairs = pd.DataFrame(pairs_data)

# To prevent display truncation if there are many rows
pd.set_option('display.max_rows', None)

print("All Combinations with Frequency > 0 and Total Revenue > 0 (same cost center only):")
print(all_pairs)

# Save to CSV for import into PowerBI
all_pairs.to_csv('all_pairs_with_cost_center.csv', index=False)

# Find All Singles in Multi-Item Baskets with Frequency and Total Revenue
single_counts = Counter()
single_revenue = defaultdict(float)
for tx_id in multi_tx_ids:
    tx_lines = multi_line_items[multi_line_items['transaction_id'] == tx_id]
    item_rev = tx_lines.groupby('item_name')['gross_revenue'].sum().to_dict()
    unique_items = set(item_rev.keys())
    for item in unique_items:
        single_counts[item] += 1
        single_revenue[item] += item_rev.get(item, 0)

# Create dataframe for singles
singles_data = []
for item, freq in sorted(single_counts.items(), key=lambda x: single_revenue[x[0]], reverse=True):
    revenue = single_revenue[item]
    if revenue > 0 and freq > 0:
        cc = item_to_cc.get(item)
        singles_data.append({
            'Item': item,
            'Cost Center': cc,
            'Frequency': freq,
            'Total Revenue': revenue
        })

all_singles_in_multi = pd.DataFrame(singles_data)

print("\nAll Single Items in Multi-Item Baskets with Frequency > 0 and Total Revenue > 0:")
print(all_singles_in_multi)

# Save to CSV for import into PowerBI
all_singles_in_multi.to_csv('all_singles_with_cost_center.csv', index=False)

All Combinations with Frequency > 0 and Total Revenue > 0 (same cost center only):
      Cost Center                                         Item1  \
0      Hospital B                                   ENTREE 4.29   
1      Hospital B                                   ENTREE 4.79   
2      Hospital B                                   ENTREE 4.99   
3      Hospital B                                 BURGER CHEESE   
4      Hospital B                                   ENTREE 4.99   
5      Hospital B                                BACON 3 SLICES   
6      Hospital B                                 SALMON ENTRÉE   
7      Hospital B                                          EGGS   
8      Hospital B                                          EGGS   
9      Hospital B                        ENT -PREMIUM SIDE 1.39   
10     Hospital B                        ENT -PREMIUM SIDE 1.39   
11     Hospital B                        ENT -PREMIUM SIDE 1.39   
12     Hospital B                             

In [7]:
all_pairs.head(100)

,Item1,Cost Center1,Item2,Cost Center2,Frequency,Total Revenue
0,ENTREE 4.29,Hospital B,SIDE 1.29,Hospital B,3659,21212.819696
1,ENTREE 4.79,Hospital B,SIDE 1.29,Hospital B,2171,14391.029803
2,ENTREE 4.99,Hospital B,SIDE 1.29,Hospital B,697,11816.789477
3,BURGER CHEESE,Hospital B,FRIES,Hospital B,1017,10889.290032
4,ENTREE 4.99,Hospital B,SIDE 1.99,Hospital B,472,8257.439672
5,BACON 3 SLICES,Hospital B,EGGS,Hospital B,2348,8250.039975
6,SALMON ENTRÉE,Hospital B,SIDE 1.29,Hospital B,851,7445.399761
7,EGGS,Hospital B,HOME FRIES,Hospital B,1961,7442.499915
8,EGGS,Hospital B,SAUSAGE PATTY,Hospital B,1548,7182.989898
9,ENT -PREMIUM SIDE 1.39,Hospital B,ENTREE 4.29,Hospital B,1155,6603.079939
